In [1]:
import pandas as pd
import numpy as np

In [2]:
from kNN import kNN

In [3]:
k = 3
seed = 42

In [4]:
random_generator = np.random.default_rng(seed)

In [5]:
df = pd.read_csv("./data/thyroidDF.csv")
df

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,target,patient_id
0,29,F,f,f,f,f,f,f,f,t,...,NaN,f,NaN,f,NaN,f,NaN,other,-,840801013
1,29,F,f,f,f,f,f,f,f,f,...,128.0,f,NaN,f,NaN,f,NaN,other,-,840801014
2,41,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,11.0,other,-,840801042
3,36,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,26.0,other,-,840803046
4,32,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,36.0,other,S,840803047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9167,56,M,f,f,f,f,f,f,f,f,...,64.0,t,0.83,t,77.0,f,NaN,SVI,-,870119022
9168,22,M,f,f,f,f,f,f,f,f,...,91.0,t,0.92,t,99.0,f,NaN,SVI,-,870119023
9169,69,M,f,f,f,f,f,f,f,f,...,113.0,t,1.27,t,89.0,f,NaN,SVI,I,870119025
9170,47,F,f,f,f,f,f,f,f,f,...,75.0,t,0.85,t,88.0,f,NaN,other,-,870119027


In [6]:
df = df[(df["age"] >= 0) & (df["age"] < 100)]
df

,age,sex,on_thyroxine,query_on_thyroxine,on_antithyroid_meds,sick,pregnant,thyroid_surgery,I131_treatment,query_hypothyroid,...,TT4,T4U_measured,T4U,FTI_measured,FTI,TBG_measured,TBG,referral_source,target,patient_id
0,29,F,f,f,f,f,f,f,f,t,...,NaN,f,NaN,f,NaN,f,NaN,other,-,840801013
1,29,F,f,f,f,f,f,f,f,f,...,128.0,f,NaN,f,NaN,f,NaN,other,-,840801014
2,41,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,11.0,other,-,840801042
3,36,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,26.0,other,-,840803046
4,32,F,f,f,f,f,f,f,f,f,...,NaN,f,NaN,f,NaN,t,36.0,other,S,840803047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9167,56,M,f,f,f,f,f,f,f,f,...,64.0,t,0.83,t,77.0,f,NaN,SVI,-,870119022
9168,22,M,f,f,f,f,f,f,f,f,...,91.0,t,0.92,t,99.0,f,NaN,SVI,-,870119023
9169,69,M,f,f,f,f,f,f,f,f,...,113.0,t,1.27,t,89.0,f,NaN,SVI,I,870119025
9170,47,F,f,f,f,f,f,f,f,f,...,75.0,t,0.85,t,88.0,f,NaN,other,-,870119027


In [7]:
qual_cols = { "age",  "TSH", "T3", "TT4", "T4U", "FTI", "TBG" }
remove_qual_cols = { "TBG" }

In [8]:
sel_qual_cols = list(qual_cols.difference(remove_qual_cols))

In [9]:
qual_na_df = df[sel_qual_cols].dropna()
qual_na_df = qual_na_df.sample(n=100, replace=False, random_state=np.random.RandomState(seed))

In [10]:
from data_split import k_fold_split

In [11]:
complete_vals_df, remove_vals_df = k_fold_split(qual_na_df, k, random_generator)
missing_vals_idxs = list(remove_vals_df.index)

real_df = pd.concat([complete_vals_df, remove_vals_df])

In [12]:
from forgetter import forget_random_col_per_sample

In [13]:
weight_map = {
    "age" : 1,
    "T3"  : 1,
    "T4U" : 1,
    "TSH" : 1,
    "TT4" : 1,
    "FTI" : 1
}

In [14]:
missing_vals_df, missing_col_map, missing_vals_idxs = forget_random_col_per_sample(remove_vals_df, weight_map, random_generator)

real_df = pd.concat([complete_vals_df, remove_vals_df])
curr_df = pd.concat([complete_vals_df, missing_vals_df])

In [15]:
predictions = {
    7 : (1, 12),
    8 : (3, 14),
    10 : (2, 11)
}

In [16]:
predictions_weights = {k: v[0] for k, v in predictions.items()}
predictions_weights

{7: 1, 8: 3, 10: 2}

In [17]:
attr_types = {
    "age" : "qualitative",
    "T3"  : "qualitative",
    "T4U" : "qualitative",
    "TSH" : "qualitative",
    "TT4" : "qualitative",
    "FTI" : "qualitative"
}

In [18]:
kNN(complete_vals_df, missing_vals_df, missing_col_map, attr_types, k, False)

,FTI,age,T3,TSH,TT4,T4U,predictions
7286,NaN,74.0,1.2,4.10,123.0,0.94,116.333333
138,149.0,63.0,2.3,0.10,NaN,1.03,130.000000
7254,145.0,46.0,NaN,2.20,116.0,0.80,1.596667
6844,NaN,58.0,2.3,2.70,91.0,0.99,115.666667
984,102.0,45.0,1.7,3.50,NaN,0.94,93.000000
6056,131.0,50.0,1.5,0.32,NaN,1.05,114.666667
149,96.0,26.0,3.6,0.60,150.0,NaN,1.040000
2549,105.0,37.0,NaN,27.00,87.0,0.84,1.966667
6012,133.0,NaN,1.7,0.44,115.0,0.87,58.666667
2387,NaN,41.0,1.9,6.00,114.0,1.00,133.333333


In [19]:
kNN(complete_vals_df, missing_vals_df, missing_col_map, attr_types, k, True)

,FTI,age,T3,TSH,TT4,T4U,predictions
7286,NaN,74.0,1.2,4.10,123.0,0.94,117.492390
138,149.0,63.0,2.3,0.10,NaN,1.03,133.316519
7254,145.0,46.0,NaN,2.20,116.0,0.80,0.760075
6844,NaN,58.0,2.3,2.70,91.0,0.99,109.619232
984,102.0,45.0,1.7,3.50,NaN,0.94,104.438008
6056,131.0,50.0,1.5,0.32,NaN,1.05,114.824632
149,96.0,26.0,3.6,0.60,150.0,NaN,1.054434
2549,105.0,37.0,NaN,27.00,87.0,0.84,2.354760
6012,133.0,NaN,1.7,0.44,115.0,0.87,38.184130
2387,NaN,41.0,1.9,6.00,114.0,1.00,132.643591


In [21]:
real_df

,FTI,age,T3,TSH,TT4,T4U
5015,113.0,83.0,1.5,2.50,91.0,0.81
2938,91.0,49.0,2.1,2.00,79.0,0.87
6058,143.0,43.0,2.5,0.68,175.0,1.23
5151,145.0,44.0,1.3,2.00,136.0,0.94
348,253.0,58.0,4.1,0.30,202.0,0.80
...,...,...,...,...,...,...
470,450.0,46.0,1.2,14.40,153.0,0.34
6701,52.0,66.0,0.3,1.70,61.0,1.18
1449,104.0,68.0,1.6,0.30,99.0,0.94
2279,76.0,45.0,2.0,1.70,71.0,0.93
